# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
import operator
import time
from sklearn.preprocessing import Imputer
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA    
from sklearn.preprocessing import LabelEncoder
# !pip install mca
# import mca
import chardet
# magic word for producing visualizations in notebook
%matplotlib inline
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score, roc_auc_score
import seaborn as sns

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [3]:
# load in the data
# azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';', dtype=str)
# customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';', dtype=str)
azdias = pd.read_csv('Udacity_AZDIAS_052018.csv', sep=';', dtype=str)
customers = pd.read_csv('Udacity_CUSTOMERS_052018.csv', sep=';', dtype=str)

In [ ]:
# Be sure to add in a lot more cells (both markdown and code) to document your
# approach and findings!

In [ ]:
azdias.head()

In [ ]:
azdias.shape

In [ ]:
azdias["LNR"].nunique()

It seems like each row has unique LNR value.

In [ ]:
customers.shape

In [ ]:
customers["LNR"].nunique() == customers.shape[0]

In [ ]:
np.intersect1d(azdias["LNR"], customers["LNR"])

## Dealing with missing data

This dataframe will help in replacing the values indicating missing data with `None` during the cleaning step.

In [ ]:
feat_info = pd.read_csv("features.csv")

In [ ]:
feat_info.head()

In [ ]:
azdias_cleaned = azdias.copy()

In [ ]:
for index, row in feat_info.iterrows():
    attribute, information_level, var_type, missing, comment = row
    if attribute in azdias_cleaned.columns:
        values = missing.replace("[","").replace("]","").split(",")
        replacement = {}
        for value in values:
            value = value.strip()
            replacement[value] = None
        azdias_cleaned.loc[:, attribute].replace(replacement, inplace=True)

In [ ]:
# Perform an assessment of how much missing data there is in each column of the
# dataset.
missing = azdias_cleaned.isnull().sum()
missing = missing[missing > 0]/(azdias.shape[0]) * 100
missing.sort_values(inplace=True)

In [ ]:
plt.hist(missing, bins=20, facecolor='c', alpha=0.75)
plt.xlabel('Percentage of missing value')
plt.ylabel('Counts')
plt.title('Distribution of missing value counts')
plt.grid(True)
plt.show()

In [ ]:
missing_20 = [col for col in azdias_cleaned.columns if (azdias[col].isnull().sum()/azdias_cleaned.shape[0]) * 100 > 20]
print(missing_20)

I have decided to drop these columns during the data cleaning step as the columns with more than 20% missing values look like outliers among all the columns as most of the columns have less than 20% missing values.

To decide on how to deal with the remaining columns with less than 20% missing values, I decided to divide the columns in two sets: one with less than or equal to x missing values and another with more than x missing values and observe the distribution of values for the columns in each set to see if there is any significant difference visible.

In [ ]:
cols_less_than_20_missing = [col for col in azdias_cleaned.columns if col not in missing_20]
missing = azdias_cleaned[cols_less_than_20_missing].isnull().sum()
missing = missing[missing > 0]/(azdias_cleaned[cols_less_than_20_missing].shape[0]) * 100
missing.sort_values(inplace=True)

In [ ]:
plt.hist(missing, bins=20, facecolor='c', alpha=0.75)
plt.xlabel('Percentage of missing value')
plt.ylabel('Counts')
plt.title('Distribution of missing value counts')
plt.grid(True)
plt.show()

In [ ]:
few_missing = azdias_cleaned[cols_less_than_20_missing][azdias_cleaned[cols_less_than_20_missing].isnull().sum(axis=1) < 20].reset_index(drop=True)
high_missing = azdias_cleaned[cols_less_than_20_missing][azdias_cleaned[cols_less_than_20_missing].isnull().sum(axis=1) >= 20].reset_index(drop=True)

In [ ]:
col_names_few = few_missing.columns

fig, axes = plt.subplots(nrows=7, ncols=2, figsize=(20,30))
sns.set(style="darkgrid")
for column in azdias_cleaned.columns[0:7]:
    sns.countplot(few_missing.loc[:, column], ax=axes[n,0])
    axes[n,0].set_title('Data with <= 20 values missing per row')
    sns.countplot(high_missing.loc[:, column], ax=axes[n,1])
    axes[n,1].set_title('Data with > 20 values missing per row')

As we can see from the above figures that there is significant difference between the distributions of the values in the rows with large number of missing values vs. low number of missing values. This suggests that the missing values are more likely to be present in rows with specific distribution of values in other columns with few missing values. This means that it is not a good idea to drop the rows with missing values and instead we need to fill them with some meaningful value. For the categorical, ordinal and interval type columns, it makes sense to fill the missing values with Mode of the column.

Column "flag indicating the former GDR/FRG" contains categorical values 'W' and 'O' which needs to be converted into binary numerical values.

For the purpose of using in PCA, I have only retained columns that have less that 30 ordinal level values during the cleaning process.

Below is the method that encodes all the cleaning steps described above. This method can be called on both population data as well as customers data.

In [ ]:
def clean_data(df_input):
    """
    Takes data frame as input. 
    Performs required cleaning steps to convert it into a dataframe useful for further analysis.
    Input:
    - df_input : Dataframe to be cleaned
    Output:
    - df : Cleaned dataframe
    """
    df = df_input.copy()
    print ("Copied")
    
    for index, row in feat_info.iterrows():
        attribute, information_level, var_type, missing, comment = row
        if attribute in df.columns:
            values = missing.replace("[","").replace("]","").split(",")
            replacement = {}
            for value in values:
                value = value.strip()
                replacement[value] = None
            df.loc[:, attribute].replace(replacement, inplace=True)
    
    print("Replaced unknown with None")
    df.replace({"-1": None, 'X': None, 'XX': None}, inplace=True)
    print ("Replaced -1, X, XX with None")
    recode = ['D19_BANKEN_DATUM', 'D19_BANKEN_OFFLINE_DATUM',
       'D19_BANKEN_ONLINE_DATUM', 'D19_GESAMT_DATUM',
       'D19_GESAMT_OFFLINE_DATUM', 'D19_GESAMT_ONLINE_DATUM',
       'D19_TELKO_DATUM', 'D19_TELKO_OFFLINE_DATUM',
       'D19_TELKO_ONLINE_DATUM', 'D19_VERSAND_DATUM',
       'D19_VERSAND_OFFLINE_DATUM', 'D19_VERSAND_ONLINE_DATUM',
       'D19_VERSI_DATUM', 'D19_VERSI_OFFLINE_DATUM',
       'D19_VERSI_ONLINE_DATUM']
    to_be_recoded = [col for col in df.columns if col in recode]
    df[to_be_recoded] = df[to_be_recoded].replace("10", "0")
    col_select = (df.isna().sum(axis=0)/df.shape[0]) <= 0.20    
    df = df.loc[:, col_select]
    print ("Retained columns with only few missing values")
    df = df.loc[:, df.nunique() <= 30]
    print ("Retained columns with limited levels")
    def fill_mode(col):
        return col.fillna(col.mode()[0])
    df = df.apply(fill_mode, axis=0)
    print ("Filled Nan with Mode")
    
    numbers = [str(x) for x in range(100)]
    for col in df.columns:
        level = 0
        for value in df[col].unique():
            if value not in numbers:
                df.loc[df[col] == value, col] = level
                print(col + " " + str(df[col].unique()))                
                print("Replaced {} with {}".format(value, level))
                level += 1
    df = df.astype(float)
    print ("Converted to numeric")
    return df

In [ ]:
common_cols = np.intersect1d(customers.columns, azdias.columns)

In [ ]:
cust = clean_data(customers[common_cols])

In [ ]:
pop = clean_data(azdias[common_cols])

In [ ]:
cust.to_csv("cust.csv", index=False, header=True, sep=";")

In [ ]:
pop.to_csv("pop.csv", index=False, header=True, sep=";")

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

In [ ]:
cust = pd.read_csv("cust.csv", sep=";")
pop = pd.read_csv("pop.csv", sep=";")

In [ ]:
cust.head()

In [ ]:
pop.head()

In [ ]:
scaler = StandardScaler()

Both the dataframes population and customers need to have same columns to be able to apply Principal Component Analysis technique with PCA model fitted using population data and then used to transform the customer data.

In [ ]:
common_cols = np.intersect1d(pop.columns, cust.columns)

In [ ]:
pop_float = pop[common_cols]

In [ ]:
pop_scaled = pd.DataFrame(scaler.fit_transform(pop_float.values), columns=pop_float.columns)

In [ ]:
pop_float.head()

In [ ]:
pop_scaled.head()

In [ ]:
cust_float = cust[common_cols].astype(float)

In [ ]:
cust_scaled = pd.DataFrame(scaler.transform(cust_float.values), columns=cust_float.columns)

In [ ]:
pca = PCA(random_state=42)

In [ ]:
pop_pca = pca.fit_transform(pop_scaled)

In PCA, I decided to retain number components that account for 80% for variance in the dataset. Below graph visualizes that it retained to 40 components which could explain 80% of variance in the data.

In [ ]:
n_components = min(np.where(np.cumsum(pca.explained_variance_ratio_)>0.8)[0]+1)

fig = plt.figure()
ax = fig.add_axes([0,0,1,1],True)
ax2 = ax.twinx()
ax.plot(pca.explained_variance_ratio_, label='Variance',)
ax2.plot(np.cumsum(pca.explained_variance_ratio_), label='Cumulative Variance',color = 'orange');
ax.set_title('n_components needed for >%80 explained variance: {}'.format(n_components));
ax.axvline(n_components, linestyle='dashed', color='black')
ax2.axhline(np.cumsum(pca.explained_variance_ratio_)[n_components], linestyle='dashed', color='black')
fig.legend(loc=(0.6,0.2));

In [ ]:
pca = PCA(n_components=n_components, random_state=42)

In [ ]:
pop_pca = pca.fit_transform(pop_float)

In [ ]:
pca.explained_variance_ratio_.sum()

In [ ]:
def get_kmeans_score(data, k):
    kmeans = KMeans(n_clusters=k, random_state=42)
    model = kmeans.fit(data)
    score = np.abs(model.score(data))
    return score

In [ ]:
scores = []
ks = list(range(2,31, 5))
for k in ks:
    scores.append(get_kmeans_score(pop_pca, k))

In [ ]:
plt.plot(ks, scores, marker='o');
plt.xlabel('K');
plt.ylabel('SSE');
plt.title('SSE vs. K');

As the reduction in Standard Squared Error of becomes very slow after 20 clusters, I decided to divide the population data into 20 clusters and proceed further.

In [ ]:
k = 20

In [ ]:
kmeans = KMeans(n_clusters=k, random_state=42, n_jobs=-1)

In [ ]:
model_pop = kmeans.fit(pop_pca)

In [ ]:
pop_pred = model_pop.predict(pop_pca) + 1

In [ ]:
cust_pca = pca.transform(cust_scaled)

In [ ]:
cust_pred = model_pop.predict(cust_pca) + 1

In [ ]:
general_prop = []
customers_prop = []
x = range(1, k+1)
for i in range(1, k+1):
    general_prop.append((pop_pred == i).sum()/len(pop_pred))
    customers_prop.append((cust_pred == i).sum()/len(cust_pred))


df_prop = pd.DataFrame({'cluster' : x, 'prop_general' : general_prop, 'prop_customers':customers_prop})

#ax = sns.countplot(x='index', y = df_general['prop_1', 'prop_2'], data=df_general )
df_prop.plot(x='cluster', y = ['prop_general', 'prop_customers'], kind='bar', figsize=(9,6))
plt.ylabel('proportion of people in each cluster')
plt.show()

From the above graph, it's visible that the customers are highly represented in the cluster number 3 and 6. Also some of the customers are represented in clusters 1, 5 and 16. Now let's try to interpret the components of the cluster which is mostly represents the consumers to understand the characteristics of the people that are assigned to that cluster.

In [ ]:
df_prop["diff"] = (df_prop["prop_customers"] - df_prop["prop_general"])/df_prop["prop_general"]

In [ ]:
df_prop["diff"]

In [ ]:
sorted_by_diff = df_prop.sort_values(by=["diff"], ascending=False)["cluster"]

Finding out the most highly representative cluster for the customers.

In [ ]:
most_likely_cust_cluster = sorted_by_diff.iloc[0]

In [ ]:
most_likely_cust_cluster

Listing out all the clusters that the customers are represented by.

In [ ]:
pd.Series(cust_pred).value_counts()

In [ ]:
cust_in_most_likely_cust_cluster = cust_pca[cust_pred == most_likely_cust_cluster]

In [ ]:
cust_in_most_likely_cust_cluster[:5]

In [ ]:
cust_in_least_likely_cust_cluster = cust_pca[cust_pred == least_likely_cust_cluster, :]

In [ ]:
def pca_weights(pca, i):
    weight_map = {}
    for counter, feature in enumerate(common_cols):
        weight_map[feature] = pca.components_[i][counter]
    
    sorted_weights = sorted(weight_map.items(), key=operator.itemgetter(1), reverse=True)
    
    return sorted_weights

I want to find out the principal component in which the customers in highly representative cluster differ significantly from the rest of the population. For that I am relying upon the distance of mean of the values of each component in terms of standard deviations and sorting them by that distance in decreasing order to see which component has highest distance from the mean.

In [ ]:
comp_significance = pd.DataFrame({"comp": range(pop_pca.shape[1]), "dist_in_std" : (cust_in_most_likely_cust_cluster.mean(axis=0) - pop_pca.mean(axis=0))/pop_pca.std(axis=0)})

In [ ]:
comp_significance.head()

In [ ]:
comp_significance["abs_dist_in_std"] = comp_significance["dist_in_std"].abs()

In [ ]:
comp_sorted = comp_significance.sort_values(by=["abs_dist_in_std"], ascending=False)

In [ ]:
comp_sorted

It is clear that component number 25 is having highest distance of values from the rest of the population. We can take a look at the weights associated with the original input variables for that component to see which variables have highest positive or negative influence on deciding the value of this component.

In [ ]:
pprint.pprint(pca_weights(pca, comp_sorted.iloc[0, 0]))

Looking at the original attributes that have highest absolute weight for the component for e.g. D19_KONSUMTYP_MAX (people who are morel likely to modern, informed or inactive in purchasing behavior), D19_VERSICHERUNGEN (people who are less likely to buy insurance), D19_BUCH_CD (people who are less likely to buy book or CD), D19_GESAMT_ANZ_12 (people who have high transactional activities in last 12 months), D19_BANKEN_DIREKT (people who are more likely to bank directly), D19_GESAMT_OFFLINE_DATUM (people who are more likely to have performed offline transactions recently), D19_HAUS_DEKO (people who are more likely to buy items related to house decoration), D19_DROGERIEARTIKEL (people who are more likely to purchase from drugstores). D19_WEIN_FEINKOST (people who are likely to buy wine). 

It feels that these traits can be observed in older people, which suggests that the most representative cluster of the population for large majority of customer base of this company is that containing relatively older people.

I have completed Part 2 and 3 in seperate notebook.